# Homework

# Making experiments

. 
## Why not Psychopy?

* Doesn't support Linux (and requires years-old deprecated libraries)
* Psychopy **just ported from Python2** (https://github.com/psychopy/psychopy/releases/tag/1.90.0)
* Requires openGL2+ (doesn't work in any Virtualbox)
* We're programmers, we don't care for GUIS with drag&drop interface anyway - GUIs have a lack of flexibility & control, limiting the scope of experimental settings
* All it's dependencies seem outdated (wxPython's newest Ubuntu/Debian installation instructions are for 2011's versions? https://wiki.wxpython.org/InstallingOnUbuntuOrDebian)
* Lack of support (their installation instructions for example are outdated by **years**: http://www.psychopy.org/installation.html)

![](psycho.png)

### ==> Let's go for an alternative!

.  

# Let's think about what we need to make our own experiment:

* Graphical User Interface
* Stimuli (Visual, auditory, ...)
* User Input
* Experiment Flow/Design
* Data logging
* (Serial) Port communication (for eg. EEG-markers)

## Graphical User Interfaces

There are quite a few libraries for experiments out there besides PsychoPy, and many of them rely on **pyGame** for their visual stimuli, providing more functionality ontop of it. But even for those which don't, the principle of how to put stuff on a screen is equal most of the time anyways.  
Let's look into pygame, to understand what's going on there:

In [ ]:
# %load pygame_1.py
import pygame

def main():

    pygame.init()

    # load and set the logo
    logo = pygame.image.load("logo32x32.jpg")
    pygame.display.set_icon(logo)
    pygame.display.set_caption("minimal program")

    # create a surface on screen that has the size of 240 x 180
    screen = pygame.display.set_mode((240,180))

    # define a variable to control the main loop
    running = True

    while running:
        # event handling, gets all event from the eventqueue
        for event in pygame.event.get():
            # only do something if the event is of type QUIT
            if event.type == pygame.QUIT:
                # change the value to False, to exit the main loop
                running = False


if __name__=="__main__":
    main()


Pygame must at first be **initialized**, before we initialize some settings for the display. Among these the *display mode* is important -- it decides the size of the actual window pygame will open in.

In [ ]:
# Creates a surface on screen that has the size of 240 x 180
screen = pygame.display.set_mode((240,180))

# Stretches this screen such that it's presented in full-screen mode
pygame.display.toggle_fullscreen()

# Creates a true full-screen Window
modes = pygame.display.list_modes(0, pygame.FULLSCREEN)
screen=pygame.display.set_mode(modes[0], pygame.FULLSCREEN)

Note that GUIs must generally perform *active waiting*, which means they must stay constantly running loop to stay active and to not close. Letting a GUI *sleep* will make it stop responding. In Python, GUIs must additionally generally be in the main *Thread*, which makes multithreaded programming in python even harder.  
This busy waiting however leads to the general structure of any game (or, for that matter, experiment): The main **game loop**

## Stimuli

We cannot have an experiment without anything to show to the user. Let's look at how to present simple stimuli using pygame. 
Anything visual must first be created as it's own *surface*, before the function *blit()* is used to copy the contents from this surface to another surface. However, doing so only draws on the back buffer where all objects are drawn before they are shown. To actually update the screen from this back buffer, we need to *flip()* the display, which will swap front and back buffer, such that we see what we drew so far, and continue drawing in the background.

### Images

In [ ]:
# %load pygame_2.py
import pygame

def main():

    pygame.init()

    # load and set the logo
    logo = pygame.image.load("logo32x32.jpg")
    pygame.display.set_icon(logo)
    pygame.display.set_caption("minimal program")

    # create a surface on screen that has the size of 240 x 180
    screen = pygame.display.set_mode((240,180))

    image = pygame.image.load("logo32x32.jpg")
    screen.blit(image, dest=(50,50))
    pygame.display.flip()
    


if __name__ == "__main__":
    main()

### Text

Note that pygame accepts only absolute coordinates on the display. To set a text precisely where you want to have it, you may need to play around with the coordinates or go for relative coordinates yourself.

In [ ]:
# %load pygame_3.py
import pygame
import time


BGCOLOR = (0, 0, 0)
SCREENSIZE = (800, 600)
FONTSIZE = SCREENSIZE[0]//20
FONTCOLOR = (255, 255, 255)

CENTER = 0
NEWLINECENTER = 1
BOTTOM = 3
TOP = 6


def get_position(screen, text_surface, where):
    text_width = text_surface.get_width()
    text_height = text_surface.get_height()
    width = screen.get_width()
    height = screen.get_height()
    if where == CENTER:
        return width/2-text_width/2, height/2-text_height/2
    elif where == NEWLINECENTER:
        return width/2-text_width/2, 5.0/8*height
    elif where == BOTTOM:
        return width/2-text_width/2, height-1.5*text_height
    elif where == TOP:
        return width/2-text_width/2, 1.0/10*height


def main():
    pygame.init()
    screen=pygame.display.set_mode(SCREENSIZE)

    screen.fill(BGCOLOR) #erase texts before
    font = pygame.font.Font(None, FONTSIZE)
    text_surface = font.render("Hello, World!", True, FONTCOLOR, BGCOLOR) #text, antialias, color, background
    posx, posy = get_position(screen, text_surface, TOP)
    screen.blit(text_surface, dest=(posx, posy))
    pygame.display.flip()

    time.sleep(5)


if __name__ == "__main__":
    main()

### Moving stimuli

In [ ]:
# %load pygame_4.py
import pygame
import time

def main():

    pygame.init()

    # create a surface on screen that has the size of 240 x 180
    screen = pygame.display.set_mode((800, 600))

    image = pygame.image.load("smiley.jpg")
    image.set_colorkey((255,255,255)) #this color is now transparent

    xpos = 50
    ypos = 50
    step_x = 10
    step_y = 10


    while True:
        if xpos > screen.get_width()-image.get_width() or xpos <= 0: step_x *= -1
        if ypos > screen.get_height()-image.get_height() or ypos <= 0: step_y *= -1
        xpos += step_x 
        ypos += step_y
        screen.fill((0,0,0))
        screen.blit(image, (xpos, ypos))
        pygame.display.flip()
        time.sleep(0.01)
        if any(event.type == pygame.QUIT for event in pygame.event.get()):
            break


if __name__ == "__main__":
    main()


In fact, if you wanted high performance, you'd work with so-called *dirty rectangles*, allowing you to only update part of the screen instead of flipping the entire screen (see eg. https://dr0id.bitbucket.io/legacy/pygame_tutorial01.html)

### Sounds

In [1]:
import pygame 

pygame.mixer.pre_init(44100, 16, 2, 4096) #frequency, size, channels, buffersize
pygame.init()
effect = pygame.mixer.Sound('beep.wav')
effect.play()

## User Input

User Input is also an *event*, just like pressing the [x] at the top-right corner. More precisely, a keypress is an event of type pygame.KEYDOWN. That means, if we encounter an event of such a type, we can check for which key it is.  
For a list of all pyGame-keys, have a look at https://docs.python.org/2.4/lib/node267.html.

In [ ]:
def wait_any_key():
    while True:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    return False
                else:
                    return event.key  #returns key numbers
            elif event.type == pygame.QUIT:
                return False


def wait_key(which):
    while True:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    return False
                elif event.key == which:
                    return True
            elif event.type == pygame.QUIT:
                return False

As the ```wait_any_key()``` and ```wait_key()```-functions are what's left from the *game loop* of the previous examples, namely the parts where the process is *actively waiting*, they are responsible for allowing the user to quit. If [ESC] or [x] is pressed, these functions return ```false```, signalling the caller that the process should be terminated.

## Putting it together

Let's use what we know so far to make a sample 'Experiment': The subject is supposed to press [left] for even numbers, and [right] for odd numbers. As we only looked at the GUI so far, we don't measure response times or save the results somehow, but simply play a sound when the subject was wrong.

In [ ]:
# %load pygame_5.py
import pygame
import time
import random


BGCOLOR = (0, 0, 0)
SCREENSIZE = (800, 600)
FONTSIZE = SCREENSIZE[0]//20
FONTCOLOR = (255, 255, 255)

CENTER = 0
NEWLINECENTER = 1
BOTTOM = 3
TOP = 6


def get_position(screen, text_surface, where):
    text_width = text_surface.get_width()
    text_height = text_surface.get_height()
    width = screen.get_width()
    height = screen.get_height()
    if where == CENTER:
        return width/2-text_width/2, height/2-text_height/2
    elif where == NEWLINECENTER:
        return width/2-text_width/2, 5.0/8*height
    elif where == BOTTOM:
        return width/2-text_width/2, height-1.5*text_height
    elif where == TOP:
        return width/2-text_width/2, 1.0/10*height


def wait_any_key():
    while True:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    return False
                else:
                    return event.key  #returns key numbers
            elif event.type == pygame.QUIT:
                return False


def wait_key(which):
    while True:
        for event in pygame.event.get():
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    return False
                elif event.key == which:
                    return True
            elif event.type == pygame.QUIT:
                return False


def main():
    pygame.mixer.pre_init(44100, 16, 2, 4096) #frequency, size, channels, buffersize
    pygame.init()
    screen=pygame.display.set_mode(SCREENSIZE)

    screen.fill(BGCOLOR)
    font = pygame.font.Font(None, FONTSIZE)
    smallfont = pygame.font.Font(None, FONTSIZE*3//4)

    intro_text = font.render("Press <left> for even numbers, and <right> else!", True, FONTCOLOR, BGCOLOR)
    posx, posy = get_position(screen, intro_text, CENTER)
    screen.blit(intro_text, (posx, posy))
    other_text = smallfont.render("Space to continue", True, FONTCOLOR, BGCOLOR)
    posx, posy = get_position(screen, other_text, NEWLINECENTER)
    screen.blit(other_text, (posx, posy))
    pygame.display.flip()

    if not wait_key(pygame.K_SPACE):
        return

    effect = pygame.mixer.Sound('beep.wav')
    num = None
    for i in range(4):
        pre_num = num
        while pre_num == num:
            num = random.randint(1, 9)
        screen.fill(BGCOLOR)
        number = font.render(str(num), True, FONTCOLOR, BGCOLOR)
        posx, posy = get_position(screen, number, CENTER)
        screen.blit(number, (posx, posy))
        pygame.display.flip()
        key = wait_any_key()
        if not key:
            return
        elif (key == pygame.K_LEFT and num % 2 != 0) or (key == pygame.K_RIGHT and num % 2 == 0):
            effect.play()


if __name__ == "__main__":
    main()


# Expyriment

While we don't show PsychoPy here (ノಠ益ಠ彡┻━┻), there are other libraries that work better and provide better compability, while being almost equally powerful. The one we're looking at today is **Expyriment**.  

It is a pure programming library (providing no GUI), is leightweight, entirely written in python, and has almost no dependencies... and can simply be installed with *pip install expyriment*. 

Expyriment builds on top of Pygame (like many similar libraries), and has a similar way of initializing a GUI, presenting stimuli, and working with user input. It further gives the possibility to explicitly define experimental designs, automatically saves everything necessary, provides useful functions like an accurate timer, and wraps the entire framework into pre-provided control structure.  

Expyriment is an all-in-one solution, and as such it is responsible for
* Stimulus presentation
* Recording of input/output events
* Communication with other devices
* Collection and preprocessing of data
* Experiment design

![](Expyriment.png)

Figure source: Krause, F. & Lindemann, O. (2014). Expyriment: A Python library for cognitive and neuroscientific experiments. Behavior Research Methods, 46(2), 416-428. doi:10.3758/s13428-013-0390-6.

All of Expyriment's modules can be used independently of each other - the structure for experimental designs is independend of the presentation software actually used, such that you can use other software for presentation of stimuli, or experimental control, ... 

## Experiment Control

Every Expyriment-experiment adheres to the main control structure, as specified by the ```control```-package.

The control-package provides access to an experiment screen, keyboard, log file, clock and device communication.

Expyriment has three Landmarks for an experiment: initialize(), start(), and end(). 
* Initialize() starts up the screen (exp.screen), a keyboard (exp.keyboard), an event log file logging stimulus presentation times & device communications, (exp.events), and an experimental clock (exp.clock)
* Start() asks for a subject ID (saved as exp.subject) and creates a data file object (exp.data)
* Between start() and end() you iterate your hierachical design (experiment $\rightarrow$ blocks $\rightarrow$ trials)
* End() ends an experiment and saves tha data and log files

In [3]:
from expyriment import design, control
from time import sleep

# create experiment object
exp = design.Experiment(name="My Experiment")

# initialize experiment object and make it active experiment
# this will show a startup screen
# it will also initialize exp.screen, exp.mouse, exp.keyboard, exp.event and exp.clock
control.initialize(exp)

sleep(2)
# this will present a subject number screen and a ready screen after initialization
# is completely finished
control.start()

sleep(2)
                        
# this will show an "ending experiment" screen and save data
control.end()

Standard output and error logging is switched off under IPython.


True

While an interactive kernel like Jupyter asks us for the mode, in a normal script you'd always be fullscreen - as long as you don't specify otherwise:

In [4]:
exp = design.Experiment(name="My Experiment")
control.set_develop_mode(True)

control.initialize(exp)
sleep(2)
control.start()
control.end()

*** DEVELOP MODE ***
Standard output and error logging is switched off under IPython.


True

## Stimuli

Expyriment contains classes for visual and auditory stimuli. 

Unlike Pygame, you don't need to flip the buffer for the stimuli to be actually shown on the screen. This is because of Expyriment's present-function:  
```present(clear=True, update=True, log_event_tag=None)```  
where ```clear``` clears the buffers before drawing, and ```update``` flips the buffer after drawing.

Furthermore, you can (and should) preload the stimuli, such that they are fully loaded upon presentation.

To show a few different stimuli in Expyriment without having to write the usual control-landmarks, let's make a context manager for the control (Please don't do that when actually working with it, this is only for presentation!):

In [5]:
from expyriment import design, control, stimuli, misc
control.set_develop_mode(True)

class StimuliDemo:
    
    def __init__(self, name):
        exp = design.Experiment(name=name)
        control.initialize(exp)
        self.exp = exp

    def __enter__(self):
        control.start()
        return self.exp

    def __exit__(self, *args):
        control.end()

*** DEVELOP MODE ***


#### Text

In [6]:
with StimuliDemo("Experiment") as exp:
    target = stimuli.TextLine(text="I am a text!", text_size=80)
    stimuli.FixCross().present()
    target.preload()
    exp.clock.wait(1000)
    target.present()
    exp.clock.wait(1000)

Standard output and error logging is switched off under IPython.


#### Sounds

In [7]:
with StimuliDemo("Experiment") as exp:
    stimuli.Tone(duration=200, frequency=2000).play()
    exp.clock.wait(1000)

Standard output and error logging is switched off under IPython.


#### Objects

In [8]:
with StimuliDemo("Experiment") as exp: 
    target = stimuli.Rectangle([50, 50], position=[20, 20], colour=misc.constants.C_RED)
    #target.preload()
    target.present()
    exp.clock.wait(1000)

Standard output and error logging is switched off under IPython.


#### Multiple objects
If we want to draw multiple objects on the same buffer, we must 
* Clear the screen for the first object, without flipping buffers
* Don't clear the screen and don't flip buffers for objects in between
* Don't clear the screen, but flip buffers, for the last object

In [9]:
with StimuliDemo("Experiment") as exp: 
    stim1 = stimuli.Circle(radius=25, colour=(255, 255, 255), position=[-100,0])
    stim2 = stimuli.Circle(radius=25, colour=(255, 255, 255), position=[100,0])
    stim1.present(clear=True, update=False)
    stim2.present(clear=False, update=True)
    exp.clock.wait(3000)

Standard output and error logging is switched off under IPython.


#### Complex objects

In [10]:
with StimuliDemo("Experiment") as exp:    
    button = stimuli.Rectangle(size=(40,20), position=(exp.screen.size[0]//2-25, 15-exp.screen.size[1]//2))
    button_text = stimuli.TextLine(text="ok", position=button.position, text_colour=misc.constants.C_WHITE)
    canvas = stimuli.BlankScreen()
    button.plot(canvas)
    button_text.plot(canvas)
    canvas.present()
    exp.clock.wait(5000)

Standard output and error logging is switched off under IPython.


## Defaults

Expyriment works with a defaults-system for all its values (font, background-color, ...). Every package of Expyriment contains a ```defaults```-object, where these values can be overwritten.

In [1]:
from expyriment import design, control, stimuli
from time import sleep
control.set_develop_mode(False)

control.defaults.window_mode = True # True corresponds to windowed
control.defaults.window_size = [800,600] # 800x600 resolution
# we are going to change the default background color for this experiment
# however this can also be changed later after initialization using exp.screen.colour()

design.defaults.experiment_background_colour = (230,230,70)

exp = design.Experiment(name="Cool Experiment")
control.initialize(exp)
control.start()
stimuli.FixCross().present()
exp.clock.wait(1000)
control.end()

Expyriment  (Python 3.6.6) 

*** NORMAL MODE ***
Standard output and error logging is switched off under IPython.


True

In [1]:
from expyriment import design, control, stimuli, misc, io
control.set_develop_mode(True)

class StimuliDemo:
    
    def __init__(self, name):
        exp = design.Experiment(name=name)
        control.initialize(exp)
        self.exp = exp

    def __enter__(self):
        control.start()
        return self.exp

    def __exit__(self, *args):
        control.end()

Expyriment  (Python 3.6.6) 

*** DEVELOP MODE ***


## User Input

Expyriment's IO module is for logging as well as user-input. It can also be used independently from other packages, to get eg. mouse presses or serial port communication.

### Keyboard.wait()

As mentioned above, every initialized experiment has access to the keyboard-object, which contains the method wait():  
```wait(keys=None, duration=None, wait_for_keyup=False, callback_function=None, process_control_events=True)```

As arguments, you can specify which keys to look for, if you want to wait for a key-release, you can specify a callback-function upon clicking, and you can set a timeout until its's no longer waited for the keypress.

The result is a tuple of (clicked character, reaction-time in ms)

In [2]:
with StimuliDemo("Experiment") as exp:  
    target = stimuli.TextLine(text="Any key to continue.", text_size=60)
    target.present()
    button, time = exp.keyboard.wait()

print(button)
print(time)
print(button == misc.constants.K_BACKSPACE)

Standard output and error logging is switched off under IPython.
32
388
False


![](ascii_table.png)

All key- and color-constants can be found in Expyriment's documentation: https://docs.expyriment.org/expyriment.misc.constants.html

### Mouse input:

In [3]:
with StimuliDemo("Experiment") as exp:  
    while True:
        exp.mouse.show_cursor()  
        button = stimuli.Rectangle(size=(50,20), position=(exp.screen.size[0]//2-35, 15-exp.screen.size[1]//2))
        button_text = stimuli.TextLine(text="close", position=button.position, text_colour=misc.constants.C_WHITE)
        canvas = stimuli.BlankScreen()
        button.plot(canvas)
        button_text.plot(canvas)
        canvas.present()
        bid, pos, _rt = exp.mouse.wait_press()
        if bid == 0 and button.overlapping_with_position(pos):
            break

Standard output and error logging is switched off under IPython.


# Experiment Design

Everybody who already participated in an experiment should be familiar with the basic structure of an experiment -- Some stimulus is shown in a multitude of similar or equal versions in a random order, sometimes mixed with distractors. Each display of a stimulus is a *trial*. Often, experiments are split up into multiple *blocks*, showing different versions of the stimuli.  
 
The general order is thus:

* There is an experiment (what's returned by ```design.experiment()```)
  * The experiments consist of blocks
    * Every block consists of trials
      * (The trials probably contain one or more stimuli)

In [4]:
%run simon_task_short

*** DEVELOP MODE ***
Standard output and error logging is switched off under IPython.


### Expyriment's Design-package
* Contains classes describing experimental structures $\Rightarrow$ Hierachy between experiment, exp. blocks. exp. trials
* Allows for between-subject-factors, that are different dependent on the subject-ID
* Can export everything to be used by other packages, can thus be used stand-alone

The design of an experiment is specified before calling ```control.start()```, such that in the actual experiment, it is enough to loop over all ```blocks``` and ```trials```, loading and presenting the stimuli from the trials:

In [5]:
from expyriment import design, control, stimuli, misc, io

exp = design.Experiment(name="My Experiment")
control.initialize(exp)

block1 = design.Block(name="Block1")
trial1 = design.Trial()
stimu1 = stimuli.Circle(50)

trial1.add_stimulus(stimu1)
block1.add_trial(trial1)
exp.add_block(block1)

control.start()

for block in exp.blocks:
    for trial in block.trials:
        for stimulus in trial.stimuli: #this is just an example, you probably wouldn't present the stimuli consecutively.b
            stimulus.present() 
            exp.clock.wait(500)
            
control.end()            

Standard output and error logging is switched off under IPython.


True

Blocks and Trials can have ```Factors```. A factor is a simple key-value-pair, that can be used to store information about a Block or Trial, such that you can restore this information throughout the experiment, and use it to display custom things and to log conditions of blocks and trials.

In [6]:
from expyriment import design, control, stimuli, misc, io
from time import sleep

exp = design.Experiment(name="My Experiment")
control.initialize(exp)

blankscreen = stimuli.BlankScreen(colour=(0,0,0))
blankscreen.preload()

block1 = design.Block(name="Block1")
block1.set_factor("Color", "green")
trial = design.Trial()
stimulus = stimuli.Circle(50, colour = misc.constants.C_GREEN)
trial.add_stimulus(stimulus)
block1.add_trial(trial, copies = 2)
exp.add_block(block1)

block2 = design.Block(name="Block2")
block2.set_factor("Color", "red")
trial = design.Trial()
stimulus = stimuli.Circle(50, colour = misc.constants.C_RED)
trial.add_stimulus(stimulus)
block2.add_trial(trial, copies = 2)
exp.add_block(block2)

control.start()
                        
for block in exp.blocks:
    print("Now we're printing", block.get_factor("Color"), "Circles.")
    for trial in block.trials:
        trial.stimuli[0].present()
        exp.clock.wait(500)
        blankscreen.present()
        exp.clock.wait(500)
        
        
control.end()

Standard output and error logging is switched off under IPython.
Now we're printing green Circles.
Now we're printing red Circles.


True

Of course, you'd normally set blocks and trials algorithmically in a loop.

In [7]:
from expyriment import design, control, stimuli, misc, io
from time import sleep

exp = design.Experiment(name="My Experiment")
control.initialize(exp)

for name, color in [["green", misc.constants.C_GREEN], ["red", misc.constants.C_RED]]:
    block = design.Block(name=name.capitalize()+" Stimuli")
    block.set_factor("Color", name)
    for where in [["left", -300], ["right", 300]]:
        t = design.Trial()
        t.set_factor("Position", where[0])
        s = stimuli.Rectangle([50, 50], position=[where[1], 0], colour=color)
        t.add_stimulus(s)
        block.add_trial(t)   
    exp.add_block(block)
        
control.start()
                        
for block in exp.blocks:
    for trial in block.trials:
        trial.stimuli[0].present()
        exp.clock.wait(1000)
        
        
control.end()

Standard output and error logging is switched off under IPython.


True

Blocks also provide the possibility to shuffle trials with it's method ```shuffle_trials(method=0, max_repetitions=None, n_segments=None)```. Note that this method even allows a possibility to not repeat too many equal trials inside a block! More on that below.

You can use the design-package alone and export the designs for other libraries:

In [8]:
from expyriment import design, control, stimuli, misc, io

exp = design.Experiment('Example experiment')
block1 = design.Block('Experimental block')
for cond in ['A', 'B', 'C']:
    trial = design.Trial()
    trial.set_factor('Condition', cond)
    block1.add_trial(trial, copies=5)
block2 = block1.copy()
block1.shuffle_trials()
block2.shuffle_trials()
exp.add_block(block1)
exp.add_block(block2)

exp.save_design('demodesign.csv')

Note that we didn't add any stimuli here. Unfortunately, Expyriment does not allow to save stimuli to a csv.

In [9]:
%cat demodesign.csv

# -*- coding: UTF-8 -*-
#exp: Example experiment
block_cnt,block_id,trial_cnt,trial_id,Condition
0,0,0,5,B
0,0,1,6,B
0,0,2,12,C
0,0,3,10,C
0,0,4,2,A
0,0,5,11,C
0,0,6,1,A
0,0,7,4,A
0,0,8,7,B
0,0,9,3,A
0,0,10,9,B
0,0,11,8,B
0,0,12,0,A
0,0,13,13,C
0,0,14,14,C
1,1,0,5,B
1,1,1,3,A
1,1,2,8,B
1,1,3,1,A
1,1,4,11,C
1,1,5,7,B
1,1,6,6,B
1,1,7,9,B
1,1,8,0,A
1,1,9,2,A
1,1,10,10,C
1,1,11,13,C
1,1,12,14,C
1,1,13,12,C
1,1,14,4,A

#### Between-Subject-Factors

In many studies, it is necessary to provide different stimuli for different subjects. In expyriment, this are simply factors that are supposed to be different **B**et**W**een **S**ubjects. Expyriment allows to differ between subjects, based on their subject-ID: The between-subjects-factor is coupled to the subject ID that was assigned when the experiment was started.

In [12]:
import pandas as pd
df = pd.read_csv("bws_study.csv", sep=';')
df

,Exp,Item,is_pract,Condition,S1,S2,S3,S4,S5,S6,S7,Answer,Index,List_Num
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MMT,1.0,1.0,1.0,Dennis kaufte einen Blumenstrauß für seine Fre...,Die Blumenladenverkäuferin hat ihm gesagt:,"Wenn es Narzissen gibt, gibt es Rosen.","Als Dennis sich umschaute, stellte er fest:",Es gab Narzissen und es gab Rosen.,Dennis ließ sich einen wunderschönen Strauß zu...,Hat Dennis Schokolade gekauft?,Nein,1.0,1.0
2,MMT,1.0,1.0,2.0,Dennis kaufte einen Blumenstrauß für seine Fre...,Die Blumenladenverkäuferin hat ihm gesagt:,"Falls es Narzissen gibt, gibt es Rosen.","Als Dennis sich umschaute, stellte er fest:",Es gab Narzissen und es gab Rosen.,Dennis ließ sich einen wunderschönen Strauß zu...,Hat Dennis Schokolade gekauft?,Nein,2.0,2.0
3,MMT,1.0,1.0,5.0,Dennis kaufte einen Blumenstrauß für seine Fre...,Die Blumenladenverkäuferin hat ihm gesagt:,"Wenn es Narzissen gibt, gibt es Rosen.","Als Dennis sich umschaute, stellte er fest:",Es gab keine Narzissen und es gab Rosen.,Dennis ließ sich einen wunderschönen Strauß zu...,Hat Dennis Schokolade gekauft?,Nein,5.0,5.0
4,MMT,1.0,1.0,6.0,Dennis kaufte einen Blumenstrauß für seine Fre...,Die Blumenladenverkäuferin hat ihm gesagt:,"Falls es Narzissen gibt, gibt es Rosen.","Als Dennis sich umschaute, stellte er fest:",Es gab keine Narzissen und es gab Rosen.,Dennis ließ sich einen wunderschönen Strauß zu...,Hat Dennis Schokolade gekauft?,Nein,6.0,6.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,NaN
6,MMT,2.0,1.0,1.0,Tim war auf dem Weg zur Drogerie.,Er bekam unterwegs noch eine SMS:,"Wenn es Toilettenpapier gibt, gibt es Seife.","Als Tim in der Drogerie ankam, sah er:",Es gab Toilettenpapier und es gab Seife.,"Tim beeilte sich, da er es eilig hatte.",Hatte Tim sein Handy mit?,Ja,10.0,2.0
7,MMT,2.0,1.0,2.0,Tim war auf dem Weg zur Drogerie.,Er bekam unterwegs noch eine SMS:,"Falls es Toilettenpapier gibt, gibt es Seife.","Als Tim in der Drogerie ankam, sah er:",Es gab Toilettenpapier und es gab Seife.,"Tim beeilte sich, da er es eilig hatte.",Hatte Tim sein Handy mit?,Ja,11.0,3.0
8,MMT,2.0,1.0,5.0,Tim war auf dem Weg zur Drogerie.,Er bekam unterwegs noch eine SMS:,"Wenn es Toilettenpapier gibt, gibt es Seife.","Als Tim in der Drogerie ankam, sah er:",Es gab kein Toilettenpapier und es gab Seife.,"Tim beeilte sich, da er es eilig hatte.",Hatte Tim sein Handy mit?,Ja,14.0,6.0
9,MMT,2.0,1.0,6.0,Tim war auf dem Weg zur Drogerie.,Er bekam unterwegs noch eine SMS:,"Falls es Toilettenpapier gibt, gibt es Seife.","Als Tim in der Drogerie ankam, sah er:",Es gab kein Toilettenpapier und es gab Seife.,"Tim beeilte sich, da er es eilig hatte.",Hatte Tim sein Handy mit?,Ja,15.0,7.0


In [13]:
from expyriment import design, control, stimuli, misc, io
from time import sleep
import pandas as pd

control.set_develop_mode(False)
control.defaults.window_mode = True
control.defaults.window_size = [800,600]
control.defaults.initialize_delay = 0

exp = design.Experiment(name="My Experiment")
control.initialize(exp)

conditions = pd.read_csv("bws_study.csv", sep=';').dropna().groupby("Condition")
for nr, data in conditions:
    block = design.Block(name="Condition"+str(int(nr)))
    block.set_factor("Condition", nr)
    for nr, sents in data.iterrows():
        t = design.Trial()
        t.set_factor("ItemNum", sents["Item"])
        for sent in ["S1", "S2", "S3", "S4", "S5", "S6", "S7"]:
            s = stimuli.TextLine(text=sents[sent], text_size=20)
            t.add_stimulus(s)
        block.add_trial(t)
    exp.add_block(block)
    
exp.add_bws_factor("FallsWennCondition", ["Wenn-Condition", "Falls-Condition"])
exp.add_bws_factor("GabGabCondition", ["GabGab-Condition", "GabnichtGab-Condition"])


control.start()

names_to_ids = {block.name:block.id for block in exp.blocks}
if exp.get_permuted_bws_factor_condition("FallsWennCondition") == "Wenn-Condition":
    to_delete = set(["Condition2", "Condition6"])
else:
    to_delete = set(["Condition1", "Condition5"])
    
if exp.get_permuted_bws_factor_condition("GabGabCondition") == "GabGab-Condition":
    to_delete.update(["Condition5", "Condition6"])
else:
    to_delete.update(["Condition1", "Condition2"])
    
for i in to_delete:
    exp.remove_block(exp.find_block(names_to_ids[i])[0])
    
print("Condition for you:", [i.name for i in exp.blocks])
print(exp.get_permuted_bws_factor_condition("GabGabCondition"))
print(exp.get_permuted_bws_factor_condition("FallsWennCondition"))
    
for block in exp.blocks:
    for trial in block.trials:
        trial.stimuli[2].present()
        exp.clock.wait(1000)
        trial.stimuli[4].present()
        exp.clock.wait(1000)
        
        
control.end()

*** NORMAL MODE ***
Standard output and error logging is switched off under IPython.
Condition for you: ['Condition5']
GabnichtGab-Condition
Wenn-Condition


True

## Measuring time

Accurate timing of stimuli and IO as well as measuring response times is crucial for all kinds of experiments, so be aware of how you measure time!  
  
A standard way of measuring time is:

In [4]:
import time
start = time.time()
time.sleep(1)
print("This took " + str(time.time()-start) + " seconds")

This took 1.0051941871643066 seconds


While generally a good idea, Python's time()-function may not be as accurate as you may think!  

For Linux and Mac, time.time()'s precision is around +-0.001 milliseconds.  
For Windows, the precision +- 16 milliseconds precision due to clock implementation problems due to process interrupts.
[source](https://stackoverflow.com/questions/1938048/high-precision-clock-in-python/38256446#38256446)

For measuring time differences, Python provides the performance-counter, that uses the most accurate measure of time your system provides (in the case of windows, one that doesn't rely on the windows clock)

In [6]:
start = time.perf_counter()
time.sleep(1)
print("This took " + str(time.perf_counter()-start) + " seconds")

This took 1.0009260789956897 seconds


But keep in mind that there are differences among CPUs and even operating systems!  
Another caveat is using time.clock() which, amongst the fact that it's deprecated since Python 3.3, behaves completely different on Windows and Unix:

```On Unix, return the current processor time as a floating point number expressed in seconds. The precision, and in fact the very definition of the meaning of “processor time”, depends on that of the C function of the same name, but in any case, this is the function to use for benchmarking Python or timing algorithms.```  
```On Windows, this function returns wall-clock seconds elapsed since the first call to this function, as a floating point number, based on the Win32 function QueryPerformanceCounter(). The resolution is typically better than one microsecond.``` [(see also)](https://stackoverflow.com/questions/25773901/understanding-time-clock-and-time-time)

In [7]:
import time
start = time.clock()
time.sleep(1)
print("This took " + str(time.clock()-start) + " seconds")

This took 0.0016410000000000036 seconds


### Pygame and timing

Pygame provides explicit information about the precision of its own stopwatch on your platform:

In [8]:
import pygame
print("Pygame's timer resolution is:", pygame.TIMER_RESOLUTION, "milliseconds.")

Pygame's timer resolution is: 10 milliseconds.


### Expyriment and timing

Expyriment's measurement of response times fortunately automatically uses the most accurate timers available.  
Further, expyriment provides its own stopwatch (```exp.clock```), that **should be used at all times in an experiment.**   
To quote their paper: ```"Since Python wraps C functions for getting the system time, the accuracy is even more precise than milliseconds (which is the unit Expyriment uses)."```

In [10]:
from expyriment import design, control, stimuli, misc, io
control.set_develop_mode(True)

class StimuliDemo:
    
    def __init__(self, name):
        exp = design.Experiment(name=name)
        control.initialize(exp)
        self.exp = exp

    def __enter__(self):
        control.start()
        return self.exp

    def __exit__(self, *args):
        control.end()

Expyriment  (Python 3.6.6) 

*** DEVELOP MODE ***


In [11]:
with StimuliDemo("Experiment") as exp:    
    target = stimuli.TextLine(text="I am a text!", text_size=80)
    target.preload()
    target.present()
    exp.clock.wait(500)

Standard output and error logging is switched off under IPython.


It is important to know if expyriment presents stimuli accurately on time.

Expyriment synchronizes visual stimulus presentation to the refresh rate of the display $\rightarrow$ time a stimulus is allgedly presented is the one you actually see it
* Pygame doesn't do that, which is why it has several milliseconds uncertainty!
* video latency is 0 ms (17ms max update interval on 60hz screen)
* audio latency is between 15 and 20 ms
* serial port latency <1ms
* in a benchmark (automatic reaction to stimulus), the response time was reliably under 2ms for visual, and under 20ms (and stable) for auditory stimuli.


To make sure that stimuli are also accurately presented precisely on time, the ```preload()``` and ```present()```-methods return the number of milliseconds they took - so you can use this time to subtract it from other waiting times!

In [12]:
letters = list("ABCDE")
with StimuliDemo("Experiment") as exp:   
    for letter in letters:
        target = stimuli.TextLine(text=letter, text_size=80)
        exp.clock.wait(500 - stimuli.FixCross().present() - target.preload())
        target.present()
        exp.clock.wait(1000)
        exp.clock.wait(1000 - stimuli.BlankScreen().present() - target.unload())

Standard output and error logging is switched off under IPython.


## Data logging

After the experiment ended using control.end(), two files will automatically be saved: 
* An event log file (```events/name_vpnr_timestamp.xpe```) that contains an automatic history of all events: 
  * A detailed description of experimantal design (including a complete listing of trials)
  * Stimulus presentation and expected IO events and device communications
  * Upon selection even more (all screen operations, full serial port communication, ...)
* A data file (```data/name_vpnr_timestamp.xpd```), containing what was manually saved during the experiment

Both files are *commented* csv-files and can be inspected with most csv-viewers (and with pandas, when explicitly skipping the commented rows!)

To add something to the data-file, you use the ```exp.data``` attribute.  
Before ```control.start()```, you can add the column names via:  
```exp.data_variable_names = ["name1", "name"]```  
Adding variables happens using 
```exp.data.add([value1, value2])```

In [13]:
# %load responsetime.py
<don't run me>

from expyriment import design, control, stimuli, misc, io
import random
control.set_develop_mode(True)

exp = design.Experiment(name="My Experiment")
control.initialize(exp)

fixcross = stimuli.FixCross()
fixcross.preload()
blankscreen = stimuli.BlankScreen()
blankscreen.preload()

b = design.Block(name="Only Block")
for i in range(10):
    waiting_time = random.randint(200, 2000)
    t = design.Trial()
    t.set_factor("waiting_time", waiting_time)
    s = stimuli.Circle(50)
    t.add_stimulus(s)
    b.add_trial(t)
exp.add_block(b)
    
    
exp.data_variable_names = ["Waiting Time", "Response Time"]
    
control.start()

for block in exp.blocks:
    for trial in block.trials:
        fixcross.present()
        exp.clock.wait(trial.get_factor("waiting_time") - trial.stimuli[0].preload())
        trial.stimuli[0].present() 
        button, rt = exp.keyboard.wait(keys=[misc.constants.K_SPACE])
        exp.data.add([trial.get_factor("waiting_time"), rt])
        
            
control.end()            

*** DEVELOP MODE ***
Standard output and error logging is switched off under IPython.


True

In [14]:
%run responsetime.py

*** DEVELOP MODE ***
Standard output and error logging is switched off under IPython.


In [15]:
%cat data/responsetime_01.xpd

#Expyriment  (Python 3.6.6), .xpd-file, coding: UTF-8
#date: Sun Jun 24 2018 15:40:29
#--EXPERIMENT INFO
#e mainfile: responsetime.py
#e sha1: None
#e modules: 
#e Experiment: My Experiment
#e no between subject factors
#e Block 0: Only Block
#e     block factors: 
#e     n trials: 10
#e     trial factors: waiting_time = [1014, 1148, 1349, 1353, 1411, 1423, 239, 277, 363, 416]
#e                    
#--SUBJECT INFO
#s id: 1
subject_id,Waiting Time,Response Time
1,416,1294
1,1411,396
1,1014,640
1,363,616
1,239,369
1,1148,201
1,1353,8330
1,277,359
1,1423,318
1,1349,263


In [16]:
import pandas as pd
FILENAME = "data/responsetime_01.xpd"

def get_first_row(filename):
    with open(filename, "r") as f:
        for num, line in enumerate(f.readlines()):
            if not line.strip().startswith("#"):
                return num
            
df = pd.read_csv(FILENAME, skiprows=get_first_row(FILENAME))
df

,subject_id,Waiting Time,Response Time
0,1,416,1294
1,1,1411,396
2,1,1014,640
3,1,363,616
4,1,239,369
5,1,1148,201
6,1,1353,8330
7,1,277,359
8,1,1423,318
9,1,1349,263


## Serial triggers

If you're working with (for example) an EEG, all you need your expyriment to do is to sent triggers to it at the right time - this is done very easily with serial ports:

In [17]:
import serial
from expyriment import design, control, stimuli, misc, io
import random
control.set_develop_mode(True)

exp = design.Experiment(name="My Experiment")
control.initialize(exp)

fixcross = stimuli.FixCross()
fixcross.preload()
blankscreen = stimuli.BlankScreen()
blankscreen.preload()

b = design.Block(name="Only Block")
for i in range(10):
    waiting_time = random.randint(200, 2000)
    t = design.Trial()
    t.set_factor("waiting_time", waiting_time)
    s = stimuli.Circle(50)
    t.add_stimulus(s)
    b.add_trial(t)
exp.add_block(b)
    
exp.serial_port = serial.Serial('COM1') # Can also be COM2, COM3, etc.    
exp.data_variable_names = ["Waiting Time", "Response Time"]
    
control.start()

for block in exp.blocks:
    for trial in block.trials:
        fixcross.present()
        exp.clock.wait(trial.get_factor("waiting_time") - trial.stimuli[0].preload())
        trial.stimuli[0].present() 
        exp.serial_port.write(chr(1)) # Send the byte 1 as a character
        button, rt = exp.keyboard.wait(keys=[misc.constants.K_SPACE])
        exp.data.add([trial.get_factor("waiting_time"), rt])
        
            
exp.serial_port.close()
control.end()            

*** DEVELOP MODE ***
Standard output and error logging is switched off under IPython.


SerialException: [Errno 2] could not open port COM1: [Errno 2] No such file or directory: 'COM1'

## Exporting data

In [18]:
# %load export_data.py
from expyriment import misc
import os

data_folder = os.path.join(os.path.dirname(__file__), "data/")
print(data_folder)
misc.data_preprocessing.write_concatenated_data(data_folder, "responsetime_", output_file="data.csv")

NameError: name '__file__' is not defined

In [19]:
%run export_data.py

/home/chris/Documents/UNI/sem_10/Scientific_Programming_Python/lectures_repository/week11-experiments/data/
** Expyriment Data Preprocessor **
found 1 subject_data sets
found 3 variables: [b'subject_id', b'Waiting Time', b'Response Time']
   reading b'responsetime_01.xpd'
write file: data.csv (30 cells in 10 rows)


In [20]:
pd.read_csv("data.csv")

# -*- coding: UTF-8 -*-
subject_id Waiting Time           Response Time
1          416                             1294
           1411                             396
           1014                             640
           363                              616
           239                              369
           1148                             201
           1353                            8330
           277                              359
           1423                             318
           1349                             263

# Putting it together

This is finally enough to code experiments!

As first example, let's look at the code for an experiment testing the **Simon effect**, which says that reaction times are faster, and reaction times more accurate, when the stimulus occurs in the same relative location as the response, even if the stimulus location is irrelevant to the task.

In two experimental tasks, participants have to respond to a rectangle on the screen, according to its color (red or green), by pressing the left or the right arrow key on the computer’s key- board. Additionally, the position of the rectangles canbe either left or right. Each trial will start with the presentation of a fixation cross for 500 ms, ollowed by the rectangle that will remain on the display until a response is given. Between trials, a blank screen is shown for 3,000 ms. Each block will contain 128 trials in  andom order. The two tasks will differ only in the mapping of responses (i.e., which button to press for which color), which will be shown to the participant as a brief  nstruction at the beginning of each block. The order of tasks will be counterb alanced over participants. The experiment has a 2×2×2×2 factorial design, with the within-subjects  actors Color (red, green), Position (left, right), and Task (left = green, left = red), as well as the between-subjects factor Task Order (left = green first,left = red first). (From the [paper](https://link.springer.com/article/10.3758%2Fs13428-013-0390-6))

In [1]:
from expyriment import design, control, stimuli, io, misc
control.set_develop_mode(False)
io.defaults.outputfile_time_stamp = False

# Create and initialize an Experiment
exp = design.Experiment("Simon Task")
control.initialize(exp)

# Define and preload standard stimuli
fixcross = stimuli.FixCross()
fixcross.preload()
blankscreen = stimuli.BlankScreen()
blankscreen.preload()
# left and right arrow keys for responses
response_keys = [misc.constants.K_LEFT, misc.constants.K_RIGHT]

# Create design
for mapping in ["left=green", "left=red"]:
    b = design.Block()
    b.set_factor("Mapping", mapping)
    for where in [["left", -300], ["right", 300]]:
        for what in [["red", misc.constants.C_RED],
                     ["green", misc.constants.C_GREEN]]:
            t = design.Trial()
            t.set_factor("Position", where[0])
            t.set_factor("Colour", what[0])
            s = stimuli.Rectangle([50, 50], position=[where[1], 0], colour=what[1])
            t.add_stimulus(s)
            b.add_trial(t, copies=2)
    b.shuffle_trials()
    exp.add_block(b)
    
exp.add_bws_factor("TaskOrder",["left=green first","left=red first"])
exp.data_variable_names = ["Mapping", "Colour", "Position", "Button", "RT"]
exp.save_design('simon_design.csv')

# Start Experiment
control.start()
if exp.get_permuted_bws_factor_condition("TaskOrder") == "left=red first":
    exp.swap_blocks(0,1)
    
for block in exp.blocks:
    stimuli.TextScreen("Instructions", block.get_factor("Mapping")).present()
    exp.keyboard.wait()
    for trial in block.trials:
        fixcross.present()
        exp.clock.wait(1000 - trial.stimuli[0].preload())
        trial.stimuli[0].present()
        button, rt = exp.keyboard.wait(keys=response_keys)
        exp.data.add([block.get_factor("Mapping"), trial.get_factor("Colour"), trial.get_factor("Position"), button, rt])

# End Experiment
control.end(goodbye_text="Thank you for participating!")
exp.save_design('simon_design_subject'+str(exp.subject)+'.csv')

Expyriment  (Python 3.6.6) 

*** NORMAL MODE ***

Python is running in an interactive shell but Expyriment wants to initialize a
fullscreen.
Switched to windows mode
Standard output and error logging is switched off under IPython.


SystemExit: 

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2971: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
% cat simon_design.csv

# -*- coding: UTF-8 -*-
#exp: Simon Task
#bws: TaskOrder=left=green first,left=red first
#bws-rand: 0
#dvn: Mapping,Colour,Position,Button,RT
block_cnt,block_id,block_Mapping,trial_cnt,trial_id,Colour,Position
0,0,left=green,0,7,green,right
0,0,left=green,1,0,red,left
0,0,left=green,2,2,green,left
0,0,left=green,3,6,green,right
0,0,left=green,4,5,red,right
0,0,left=green,5,3,green,left
0,0,left=green,6,4,red,right
0,0,left=green,7,1,red,left
1,1,left=red,0,1,red,left
1,1,left=red,1,0,red,left
1,1,left=red,2,7,green,right
1,1,left=red,3,2,green,left
1,1,left=red,4,3,green,left
1,1,left=red,5,5,red,right
1,1,left=red,6,4,red,right
1,1,left=red,7,6,green,right

In [3]:
% cat data/ipykernel_launcher_20.xpd

cat: data/ipykernel_launcher_72.xpd: No such file or directory


In [4]:
% cat events/ipykernel_launcher_20.xpe

cat: events/ipykernel_launcher_72.xpe: No such file or directory


# More resources

* If you also want to know something about PsychoPy, we added last week's *Basic Programming in Python* Lecture about experiments, including also Psychopy
* First address for Expyriment: Their [website](http://www.expyriment.org/).
* A nice introduction is given by their paper: Krause, F. & Lindemann, O. (2014). Expyriment: A Python library for cognitive and neuroscientific experiments. Behavior Research Methods, 46(2), 416-428. [doi:10.3758/s13428-013-0390-6](https://link.springer.com/article/10.3758%2Fs13428-013-0390-6).
* It's always, always, always helpful to look at the docs! They provide a nice Overview, as well as the API reference, under [https://docs.expyriment.org/](https://docs.expyriment.org/)
* A nice starting point if you want to make your own experiments is also their experiment-stash: [https://github.com/expyriment/expyriment-stash](https://github.com/expyriment/expyriment-stash)

# Appendix
## Shuffling

Shuffling your conditions is harder as it seems at first: Imagine you merge two experiments into one, additionally providing distractors. It may be unwanted to have more than two trial of the same condition behind each other. Further, you may not want the first experiment to be present only in the first half of the experiment, leaving only the other one and the distractors for the second half.

In [ ]:
import random
from functools import reduce

def condition(ls):
    """returns False if two consecutive list-elements start with the same letter, else True"""
    return reduce(lambda x,y: False if x is False or x[0]==y[0] else y, ls) != False

def generate_lists(num_elems):
    l1 = list(["A"+str(i) for i in range(num_elems)])
    l2 = list(["B"+str(i) for i in range(num_elems)])
    l3 = list(["C"+str(i) for i in range(num_elems)])
    return l1+l2+l3

together = generate_lists(10)
random.shuffle(together)

print(together)
condition(together)

The easiest approach to do this is to just test the conditions you want in a function, and to shuffle until this condition is met. But beware, this has a theoretical runtime of $\mathcal{O}(\infty)$, and thus may run indefinitely long or not even lead to any result at all! The more restrictions you have to your shuffle and the more items the lists have, the less likely that any shuffle will even be 

In [ ]:
random.seed(1)

toegether = generate_lists(10)

while not condition(together):
    random.shuffle(together)

print(together)

In [ ]:
together = generate_lists(15)

while not condition(together):
    random.shuffle(together)

print(together)

A better approach in this situation may be *divide and conquer*, provided the problem you have can easily be divided.

In [ ]:
def generate_lists2(num_elems):
    l1 = list(["A"+str(i) for i in range(num_elems)])
    l2 = list(["B"+str(i) for i in range(num_elems)])
    l3 = list(["C"+str(i) for i in range(num_elems)])
    for ls in [l1, l2, l3]:
        random.shuffle(ls)
    return l1, l2, l3

NUM_ELEMS = 30
OKAY_ELEMS = 5
l1, l2, l3 = generate_lists2(NUM_ELEMS)
tmp_lists = []

for i in range(NUM_ELEMS//OKAY_ELEMS):
    tmp = l1[i*OKAY_ELEMS: (i+1)*OKAY_ELEMS] + l2[i*OKAY_ELEMS: (i+1)*OKAY_ELEMS] + l3[i*OKAY_ELEMS: (i+1)*OKAY_ELEMS]
    while not condition(tmp):
        random.shuffle(tmp)
    tmp_lists.append(tmp)
                
for i in range(len(tmp_lists)-1):
    while not condition([tmp_lists[i][-1]]+[tmp_lists[i+1][0]]):
        tmp_lists[i+1] = tmp_lists[i+1][1:] + [tmp_lists[i+1][0]]
    
final_list = reduce(lambda x,y: x+y, tmp_lists)

print(final_list)

Another possibility is to just drop the idea of "shuffling until you find one" idea completely, and build the desired sequence one at a time:

In [ ]:
import numpy
lists = generate_lists2(30)

#print(lists)
#print(list(numpy.where(list_allowed))[0])


amdone = False
while not amdone:
    try:
        list_allowed = [True, True, True]
        indices = [0, 0, 0]
        merged = []
        for _ in range(sum(len(i) for i in lists)):
            next_val = random.choice(list(numpy.where(list_allowed))[0])
            merged.append(lists[next_val][indices[next_val]])
            indices[next_val] += 1
            list_allowed = [True, True, True]
            list_allowed[next_val] = False
            for i in range(len(indices)):
                if indices[i] >= len(lists[i]):
                    list_allowed[i] = False
    except IndexError:
        pass
    else:
        amdone = True
    
    
print(merged)

How expyriment shuffles: https://docs.expyriment.org/expyriment.design.Block.html#expyriment.design.Block.shuffle_trials